In [ ]:
import pennylane as qml
from pennylane import numpy as np

Recall that in the usual representation of a quantum system in a pure state as a single ket, or a linear combination of kets, a gate acting on the state is represented as $U\ket{\psi}$, where $U$ is a linear operator and $\ket{\psi}$ represents the pure state.

In the representation of possibly mixed states as a linear operator, the same operation corresponds to mapping the density operator $\rho$ to $U\rho U^{\dagger}$, so that $\ket{\psi}\bra{\psi}$ gets mapped to $U\ket{\psi}\bra{\psi}U^{\dagger}$.

We see that in terms of the representation of the mixture state as $\sum_i p_i\ket{i}\bra{i}$, the probability gets factored out as $U(\sum_i p_i\ket{i}\bra{i})U^{\dagger}=\sum_i p_i U\ket{i}\bra{i}U^{\dagger}$, corresponding to the notion that the gate acted in the same manner regardless to which state the system was originally in.

At the end of the day, we need to measure the quantum system for our formalism to have any utility. Let us review measurement on a pure state. A measurement with respect to an observable is represented as a **Hermitian operator**, so it can be represented as a summation $\sum_k \lambda_k \ket{k}\bra{k}$ where the $\ket{k}$ are orthonormal. Measurement tells us one of the eigenvalues of the system, and the immediate post-measurement state of the system is the original state projected to the subspace spanned by the eigenvectors having the measured eigenvalue, then renormalized to unit magnitude.

In concrete algebraic terms, we may express the spanned subspace as a sum of component orthonormal basis vectors (renormalized so that the sum is unit); express the pre-measurement state in the same basis, then perform the product. The result is the post-measurement state up to normalization. We perform renormalization up to global phase (which is meaningless) by dividing it with its own magnitude; the magnitude is exactly the root of the probability that the pre-measurement state collapsed to the current one, and is the sum of the squared coefficients that the pre-measurement state has of the basis vectors in the spanned eigenspace.

With a mixed state expressed as a weighted sum of states, the probability of our observation is simply the weighted probability with respect to each component (a pure state) of the mixture.

Similarly, the post-measurement mixed state is simply the weighted sum of the post-measurement pure states.

## N.2.1a

In [ ]:
def apply_gate_mixed(rho,U):
    
    """
    Args:
        rho: (np.array(array[complex]): The density matrix of the input state
        U (np.array(array[complex])): A matrix representing the unitary gate U to be applied.
        
    Returns:
        (np.array([array[complex]])): The density matrix of the output state.
    """

    ################
    #YOUR CODE HERE#
    ################
    
    return U @ rho @ np.conj(U).T # Return the density matrix

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_mixed(rho,U).round(2))


## N.2.1b

In [ ]:
dev = qml.device('default.mixed', wires=1) # Define a default.mixed device for 1 qubit

@qml.qnode(dev)
def apply_gate_circuit(rho,U):

    """
    Args:
        rho: (np.array(array[complex]): The density matrix of the input state.
        U (np.array(array[complex])): A matrix representing the unitary gate U to be applied.
        
    Returns:
        (np.array([array[complex]])): The density matrix of the output state.
    """
    
    # Prepare a state with density operator rho
    qml.QubitDensityMatrix(rho, wires=0)
    # Apply the unitary U
    qml.QubitUnitary(U, wires=0)
    
    return qml.state() # Return the density operator after applying U

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_circuit(rho,U).round(2))


## N.2.2a

In [ ]:
def eigenprojectors(obs):
    
    """
    Args:
        obs (np.array([array[complex]])): A Hermitian operator representing a quantum observable.
        
    Returns:
        (np.array(array[array[complex]])): An array containing the eigenprojectors of the observable.
        Therefore, it is an array that contains matrices.
    """

    ################
    #YOUR CODE HERE#
    ################

    eigenvectors = np.linalg.eigh(obs)[1]
    
    projectors = np.ndarray([ np.outer(eigenvectors[:,i],np.conj(eigenvectors[:,i])) for i in range(eigenvectors.shape[1]) ])
    
    return projectors


# N.2.2b

In [ ]:
def outcome_probs(rho, B):

    """
    Args:
        rho (np.array([array[complex]])): The density matrix of the state before measurement
        B (np.array([array[complex]])): Matrix representation of the measured observable
    Returns:
        (np.array(float)): List of measurement probabilities, in no particular order.
    """
    
    eigen_projectors = eigenprojectors(B)
    
    prob_list = np.array([ np.trace(projector @ rho) for projector in eigen_projectors ], dtype=np.float64)
    
    return prob_list

rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliY(0))

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(B))
print("Measurement probabilities {}".format(outcome_probs(rho,B).round(2)))



## N.2.2c

In [ ]:
dev = qml.device('default.mixed', wires = 1)

@qml.qnode(dev)
def mixed_probs_circuit(rho, obs):
    """
    Prepares a state with density matrix rho and returns measurement
    probabilities associated to the observable obs.

    Args:
        rho (np.array[array[complex]]): The density matrix we need to prepare
        obs: (pennylane.operation): A PennyLane hermitian operator 

    Returns:
        np.array(float): The measurement probabilities as required
    """
    
    # Prepare the density matrix
    qml.QubitDensityMatrix(rho, wires=0)
    
    return qml.probs(op=obs)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliY(0)

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(qml.matrix(B)))
print("Measurement probabilities {}".format(mixed_probs_circuit(rho,B))) 


## N.2.3a

In [ ]:
def mixed_expval(rho, B):

    """
    Returns the expectation value of the observable B on state rho

    Args: 
        rho (np.array(array(float)): A numpy matrix representing a density matrix
        B (np.array(array(complex))): A numpy matrix representing an observable

    Returns:
        float: The expectation value as required
    """

    ##################
    # YOUR CODE HERE #
    ##################
    
    return np.trace(rho @ B)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliZ(0))

print("State: {}".format(rho))
print("Observable: {}".format(B))
print("Expectation value: {}".format(mixed_expval(rho,B)))


## N.2.3b

In [ ]:
dev = qml.device('default.mixed', wires = 1)

@qml.qnode(dev)
def expval_circuit(rho,obs):

    """
    Returns the expectation value of the observable obs on state rho

    Args: 
        rho (np.array(array(float)): A numpy matrix representing a density matrix
        B (pennylane.operation): A pennylane observable

    Returns:
        np.tensor: A numpy tensor with the expectation value as required
    """

    ##################
    # YOUR CODE HERE #
    ##################
    qml.QubitDensityMatrix(rho, wires=0)
    return qml.expval(obs)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliZ(0)

print("State: {}".format(rho))
print("Observable: {}".format(qml.matrix(B)))
print("Expectation value: {}".format(expval_circuit(rho,B)))
